In [1]:
import nltk
from nltk import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
import pandas as pd
import json

In [2]:
drink_sentences = "sentences_drink.txt"

In [3]:
sentences_df = pd.read_csv(drink_sentences, header=0)
sentences_df.tail(10)

,'intent':'order_drink'
5,Mohl bych dostat další sklenici prosím?
6,Mohu se podívat na vinný lístek prosím?
7,Mohu vám přinést něco k pití?
8,Dám si kávu
9,Mám rád kozel
10,Chtěl bych kávu.
11,ja si Dám Pilsner Urquell
12,pivo prosím
13,Mám rád kávu
14,Mám rád kolu


In [4]:
def listing(sentence):
    return [sentence]

def tokenize_sentence(sentence):
    tk = word_tokenize(sentence[0])
    return tk

def tagger(sentence, entity, label):
    
    detkn = TreebankWordDetokenizer()
        
    try: 
        
        if entity in sentence:
            
            out = []
            
            index_selector = sentence.index(entity)
            for counter, word in enumerate(sentence,0):
                
                if counter != index_selector:
                    
                    out.append(word)
                elif counter == index_selector:
                    matcher = '['+ word +']'
                    out.append(matcher)
                    if sentence[-1] != word:
                        #[Dám, si, kávu]
                        out.append(sentence[-1])
                    index_finder = out.index(matcher)
                    out.insert(index_finder+1, (label))
                    sentence_out = detkn.detokenize(out)
                    
            return sentence_out
        
        else:
            
            return sentence
    
    except: 
        return sentence
        
def untokenizer(sentence):
    if isinstance(sentence, list):
        detkn = TreebankWordDetokenizer()
        sentence = detkn.detokenize(sentence)
        return sentence  
    else:
        return sentence

In [5]:
sentences_df["'intent':'order_drink'"] = sentences_df["'intent':'order_drink'"].apply(listing)
sentences_df["'intent':'order_drink'"] = sentences_df["'intent':'order_drink'"].apply(tokenize_sentence)


In [6]:
entities = ['pivo', 'kávu', 'kolu', 'kozel']
label = '(drink)'
for entity in entities:
    sentences_df["'intent':'order_drink'"] = sentences_df["'intent':'order_drink'"].apply(tagger,entity= entity, label=label)
  
    

In [7]:
sentences_df

,'intent':'order_drink'
0,jedno [pivo] (drink) prosim
1,Dáte si ješte jedno [pivo] (drink)?
2,"[Chtěl, bych, dolít, prosím, !]"
3,"[Chtěl, bych, láhev, vína, prosím, .]"
4,"[Mohl, bych, dostat, sklenku, perlivé, vody, ?]"
5,"[Mohl, bych, dostat, další, sklenici, prosím, ?]"
6,"[Mohu, se, podívat, na, vinný, lístek, prosím, ?]"
7,"[Mohu, vám, přinést, něco, k, pití, ?]"
8,Dám si [kávu] (drink)
9,Mám rád [kozel] (drink)


In [8]:
sentences_df["'intent':'order_drink'"] = sentences_df["'intent':'order_drink'"].apply(untokenizer)

In [9]:
sentences_df

,'intent':'order_drink'
0,jedno [pivo] (drink) prosim
1,Dáte si ješte jedno [pivo] (drink)?
2,Chtěl bych dolít prosím!
3,Chtěl bych láhev vína prosím.
4,Mohl bych dostat sklenku perlivé vody?
5,Mohl bych dostat další sklenici prosím?
6,Mohu se podívat na vinný lístek prosím?
7,Mohu vám přinést něco k pití?
8,Dám si [kávu] (drink)
9,Mám rád [kozel] (drink)


In [10]:
# write to json format the sentences
# generate the .md

#### the data structure
[
    {
        "intent": "order_drink",
        "text": "Chtěl bych kávu",
        "index": 0
    }
]

In [21]:
dataset = []
for i, sentence in enumerate(sentences_df["'intent':'order_drink'"],0):
    dataset.append(json.dumps({"intent": "order_drink",
                              "text":sentences_df["'intent':'order_drink'"][i],
                               "index": i
                              }, ensure_ascii=False).encode('utf-8'))

In [27]:
dataset

[b'{"intent": "order_drink", "text": "jedno [pivo] (drink) prosim", "index": 0}',
 b'{"intent": "order_drink", "text": "Da\xcc\x81te si jes\xcc\x8cte jedno [pivo] (drink)?", "index": 1}',
 b'{"intent": "order_drink", "text": "Cht\xc4\x9bl bych dol\xc3\xadt pros\xc3\xadm!", "index": 2}',
 b'{"intent": "order_drink", "text": "Cht\xc4\x9bl bych l\xc3\xa1hev v\xc3\xadna pros\xc3\xadm.", "index": 3}',
 b'{"intent": "order_drink", "text": "Mohl bych dostat sklenku perliv\xc3\xa9 vody?", "index": 4}',
 b'{"intent": "order_drink", "text": "Mohl bych dostat dal\xc5\xa1\xc3\xad sklenici pros\xc3\xadm?", "index": 5}',
 b'{"intent": "order_drink", "text": "Mohu se pod\xc3\xadvat na vinn\xc3\xbd l\xc3\xadstek pros\xc3\xadm?", "index": 6}',
 b'{"intent": "order_drink", "text": "Mohu v\xc3\xa1m p\xc5\x99in\xc3\xa9st n\xc4\x9bco k pit\xc3\xad?", "index": 7}',
 b'{"intent": "order_drink", "text": "Da\xcc\x81m si [ka\xcc\x81vu] (drink)", "index": 8}',
 b'{"intent": "order_drink", "text": "Ma\xcc\x81m ra

In [29]:
dataset_nn = []
for i, sentence in enumerate(sentences_df["'intent':'order_drink'"],0):
    dataset_nn.append({"intent": "order_drink",
                    "text":sentences_df["'intent':'order_drink'"][i],
                    "index": i
                    })

In [31]:
dataset_nn[4]

{'intent': 'order_drink',
 'text': 'Mohl bych dostat sklenku perlivé vody?',
 'index': 4}

In [33]:
# the dataset_nn I have to write to json later, and this one will be used to be converted to md then NN tagged

with open('training_data.json', 'w', encoding='utf8') as json_file:
    json.dump(dataset_nn, json_file, ensure_ascii=False, indent=4)